In [1]:
import pandas as pd 
import os
from scipy.io import loadmat
import numpy as np

import os
import numpy as np

path = r'C:\Users\user\Documents\Jupyter\DB2\DB2 Matlab\media\DB2_All'
S_range = range(1, 41)
E_range = range(1, 4)

emg_data = []
stimulus_data = []
repetition_data = []
subject = []

for i in S_range:
    for j in E_range:
        filename = f"S{i}_E{j}_A1.mat"
        file_path = os.path.join(path, filename)
        
        if os.path.exists(file_path):
            data = loadmat(file_path)
            num_samples = data['emg'].shape[0]
            emg_data.append(data['emg'])
            stimulus_data.append(data['stimulus'])
            repetition_data.append(data['repetition'])
            for k in range(1,num_samples + 1):
                subject.append(i)

emg_data = np.concatenate(emg_data, axis=0)
stimulus_data = np.concatenate(stimulus_data, axis=0)
repetition_data = np.concatenate(repetition_data, axis=0)
subject = np.array(subject)

print("EMG Data Shape:", emg_data.shape)
print("Stimulus Data Shape:", stimulus_data.shape)
print("Repetition Data Shape:", repetition_data.shape)
print("Subject Shape:", subject.shape)


EMG Data Shape: (207713115, 12)
Stimulus Data Shape: (207713115, 1)
Repetition Data Shape: (207713115, 1)
Subject Shape: (207713115,)


In [2]:
subject = subject.reshape(-1,1)

In [3]:
final = np.hstack((emg_data,stimulus_data,repetition_data,subject))
final

array([[-2.85120973e-06,  3.35526079e-06,  1.59886599e-06, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-4.69773840e-06,  3.35521236e-06,  1.43044065e-06, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-2.68373401e-06,  4.02655951e-06,  9.26316488e-07, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       ...,
       [ 1.51801996e-05, -1.81108771e-05, -1.51064314e-05, ...,
         4.90000000e+01,  6.00000000e+00,  4.00000000e+01],
       [ 3.36442426e-05, -1.42503286e-05,  2.28274967e-05, ...,
         4.90000000e+01,  6.00000000e+00,  4.00000000e+01],
       [ 1.21601652e-05, -4.51505548e-06,  3.60875856e-05, ...,
         4.90000000e+01,  6.00000000e+00,  4.00000000e+01]])

In [4]:
print(np.unique(final[:,-3]))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49.]


In [5]:
print(np.unique(final[:,-1]))

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40.]


In [6]:
def keep_rows_with_values(array, column_index, values_to_remove):
    mask = np.isin(array[:, column_index], values_to_remove)
    filtered_array = array[mask]
    filtered_array = filtered_array.astype(float)
    return filtered_array

In [7]:
train = 0
test = 0
repetition_data=0
stimulus_data=0
emg_data=0
sunject=0

In [8]:
train = keep_rows_with_values(final,13,[1,3,4,6])

In [9]:
test = keep_rows_with_values(final,13,[2,5])

In [10]:
train = train.astype(np.float16)

In [11]:
test = test.astype(np.float16)

In [16]:
train_final_list = []
train_final = np.empty((0,15),dtype=np.float16)
max_samples = 10000

for i in np.unique(train[:, -3]):
    selection = train[train[:, -3] == i]
    for j in [1, 3, 4, 6]:
        new_selection = selection[selection[:, -2] == j]
        for k in np.unique(train[:, -1]):
            k_selection = new_selection[new_selection[:, -1] == k]
            if len(k_selection) >= max_samples:
                data = k_selection[:max_samples]
            else:
                num_samples = k_selection.shape[0]
                padding = np.zeros((max_samples - num_samples, 15), dtype=np.float16)
                data = np.vstack((k_selection, padding))
            print(data.shape)
            print(data)
            train_final_list.append(data)

train_final = np.vstack(train_final_list)

print("Train Final Shape:", train_final.shape)

(10000, 15)
[[ 1.043e-05 -4.292e-06  3.040e-06 ...  1.000e+00  1.000e+00  1.000e+00]
 [ 1.180e-05 -3.934e-06  2.682e-06 ...  1.000e+00  1.000e+00  1.000e+00]
 [ 1.180e-05 -8.941e-07  4.053e-06 ...  1.000e+00  1.000e+00  1.000e+00]
 ...
 [-1.311e-05 -8.047e-06 -7.093e-06 ...  1.000e+00  1.000e+00  1.000e+00]
 [-1.159e-04 -2.718e-05 -9.894e-06 ...  1.000e+00  1.000e+00  1.000e+00]
 [-2.003e-04 -3.088e-05 -8.941e-06 ...  1.000e+00  1.000e+00  1.000e+00]]
(10000, 15)
[[ 4.852e-05  3.082e-05  9.775e-06 ...  1.000e+00  1.000e+00  2.000e+00]
 [ 2.229e-05  6.020e-06  4.947e-06 ...  1.000e+00  1.000e+00  2.000e+00]
 [ 5.841e-06 -2.384e-06  2.027e-06 ...  1.000e+00  1.000e+00  2.000e+00]
 ...
 [ 8.941e-06 -3.844e-05  1.103e-05 ...  1.000e+00  1.000e+00  2.000e+00]
 [-6.318e-06 -2.551e-05  7.689e-06 ...  1.000e+00  1.000e+00  2.000e+00]
 [-1.889e-05 -1.830e-05  4.172e-06 ...  1.000e+00  1.000e+00  2.000e+00]]
(10000, 15)
[[-7.87e-06 -1.55e-06 -1.19e-07 ...  1.00e+00  1.00e+00  3.00e+00]
 [-6.08e-

(10000, 15)
[[ 1.79e-06 -4.65e-06  2.74e-06 ...  1.00e+00  3.00e+00  1.00e+00]
 [ 7.75e-07 -4.29e-06  3.10e-06 ...  1.00e+00  3.00e+00  1.00e+00]
 [-3.04e-06 -3.46e-06  1.61e-06 ...  1.00e+00  3.00e+00  1.00e+00]
 ...
 [ 4.71e-06 -4.53e-06 -4.83e-06 ...  1.00e+00  3.00e+00  1.00e+00]
 [ 3.52e-06 -1.73e-05 -2.15e-06 ...  1.00e+00  3.00e+00  1.00e+00]
 [-4.77e-07 -3.71e-05  5.96e-08 ...  1.00e+00  3.00e+00  1.00e+00]]
(10000, 15)
[[2.241e-05 8.565e-05 1.580e-05 ... 1.000e+00 3.000e+00 2.000e+00]
 [3.028e-05 5.996e-05 1.794e-05 ... 1.000e+00 3.000e+00 2.000e+00]
 [3.028e-05 5.245e-06 1.764e-05 ... 1.000e+00 3.000e+00 2.000e+00]
 ...
 [5.096e-05 6.813e-05 1.323e-05 ... 1.000e+00 3.000e+00 2.000e+00]
 [6.539e-05 7.939e-05 2.331e-05 ... 1.000e+00 3.000e+00 2.000e+00]
 [5.329e-05 7.230e-05 2.855e-05 ... 1.000e+00 3.000e+00 2.000e+00]]
(10000, 15)
[[ 3.219e-06 -8.345e-07  4.172e-06 ...  1.000e+00  3.000e+00  3.000e+00]
 [ 3.457e-06 -1.729e-06  2.861e-06 ...  1.000e+00  3.000e+00  3.000e+00]
 [

(10000, 15)
[[ 1.848e-06  7.331e-06 -5.960e-08 ...  1.000e+00  4.000e+00  1.000e+00]
 [-1.788e-07  5.960e-06 -2.265e-06 ...  1.000e+00  4.000e+00  1.000e+00]
 [-3.219e-06  5.484e-06 -4.113e-06 ...  1.000e+00  4.000e+00  1.000e+00]
 ...
 [-5.949e-05  8.881e-06  3.099e-06 ...  1.000e+00  4.000e+00  1.000e+00]
 [-8.571e-05  1.389e-05  4.172e-07 ...  1.000e+00  4.000e+00  1.000e+00]
 [-8.535e-05  2.366e-05 -2.265e-06 ...  1.000e+00  4.000e+00  1.000e+00]]
(10000, 15)
[[ 8.3745e-05  1.3101e-04  9.6142e-05 ...  1.0000e+00  4.0000e+00
   2.0000e+00]
 [ 1.1396e-04  6.7174e-05  1.0353e-04 ...  1.0000e+00  4.0000e+00
   2.0000e+00]
 [ 4.0114e-05 -2.2948e-05  6.4909e-05 ...  1.0000e+00  4.0000e+00
   2.0000e+00]
 ...
 [ 1.9419e-04  1.1778e-04  4.4823e-05 ...  1.0000e+00  4.0000e+00
   2.0000e+00]
 [ 1.3900e-04  1.0657e-04  5.9247e-05 ...  1.0000e+00  4.0000e+00
   2.0000e+00]
 [ 6.0141e-05  1.1611e-04  7.0691e-05 ...  1.0000e+00  4.0000e+00
   2.0000e+00]]
(10000, 15)
[[-4.655e-05 -1.186e-05 -3.0

(10000, 15)
[[-3.338e-06  1.967e-06 -3.994e-06 ...  1.000e+00  6.000e+00  1.000e+00]
 [-2.325e-06 -4.172e-07 -5.841e-06 ...  1.000e+00  6.000e+00  1.000e+00]
 [-1.490e-06 -4.768e-06 -6.139e-06 ...  1.000e+00  6.000e+00  1.000e+00]
 ...
 [ 1.788e-06  2.277e-05  5.603e-06 ...  1.000e+00  6.000e+00  1.000e+00]
 [-8.941e-07  3.195e-05  4.947e-06 ...  1.000e+00  6.000e+00  1.000e+00]
 [-2.384e-06  4.101e-05  6.258e-06 ...  1.000e+00  6.000e+00  1.000e+00]]
(10000, 15)
[[ 8.7440e-05 -1.1486e-04 -4.3511e-06 ...  1.0000e+00  6.0000e+00
   2.0000e+00]
 [ 6.8963e-05 -8.5473e-05 -1.5974e-05 ...  1.0000e+00  6.0000e+00
   2.0000e+00]
 [ 5.4538e-05 -4.5717e-05 -1.9133e-05 ...  1.0000e+00  6.0000e+00
   2.0000e+00]
 ...
 [ 2.3627e-04  1.1885e-04  1.7846e-04 ...  1.0000e+00  6.0000e+00
   2.0000e+00]
 [ 2.6298e-04  1.3995e-04  2.3007e-04 ...  1.0000e+00  6.0000e+00
   2.0000e+00]
 [ 2.0814e-04  8.4758e-05  2.3317e-04 ...  1.0000e+00  6.0000e+00
   2.0000e+00]]
(10000, 15)
[[ 1.335e-05 -4.947e-06  3.8

(10000, 15)
[[-1.054e-04 -1.687e-05 -2.325e-06 ...  2.000e+00  1.000e+00  1.000e+00]
 [-1.465e-04 -2.027e-05 -8.225e-06 ...  2.000e+00  1.000e+00  1.000e+00]
 [-1.327e-04 -1.752e-05 -1.609e-05 ...  2.000e+00  1.000e+00  1.000e+00]
 ...
 [ 2.056e-05  3.934e-05 -2.980e-07 ...  2.000e+00  1.000e+00  1.000e+00]
 [ 1.371e-05  3.868e-05 -4.768e-07 ...  2.000e+00  1.000e+00  1.000e+00]
 [ 1.603e-05  3.183e-05  5.364e-07 ...  2.000e+00  1.000e+00  1.000e+00]]
(10000, 15)
[[ 2.038e-05  1.371e-05  4.768e-07 ...  2.000e+00  1.000e+00  2.000e+00]
 [ 2.068e-05  1.353e-05  8.345e-07 ...  2.000e+00  1.000e+00  2.000e+00]
 [ 1.836e-05  1.240e-05 -2.205e-06 ...  2.000e+00  1.000e+00  2.000e+00]
 ...
 [-1.472e-05  6.557e-06  3.338e-06 ...  2.000e+00  1.000e+00  2.000e+00]
 [-2.027e-05  1.013e-05  1.013e-06 ...  2.000e+00  1.000e+00  2.000e+00]
 [-1.961e-05  7.391e-06 -3.874e-06 ...  2.000e+00  1.000e+00  2.000e+00]]
(10000, 15)
[[-1.490e-06 -2.742e-06 -5.960e-07 ...  2.000e+00  1.000e+00  3.000e+00]
 [-

...

(10000, 15)
[[-9.537e-07  1.967e-06  2.015e-05 ...  4.900e+01  1.000e+00  1.000e+00]
 [ 1.013e-06  7.749e-07  1.830e-05 ...  4.900e+01  1.000e+00  1.000e+00]
 [ 2.205e-06  2.384e-07  1.544e-05 ...  4.900e+01  1.000e+00  1.000e+00]
 ...
 [ 8.744e-05  2.170e-05 -2.789e-04 ...  4.900e+01  1.000e+00  1.000e+00]
 [ 7.367e-05  2.027e-06 -3.293e-04 ...  4.900e+01  1.000e+00  1.000e+00]
 [ 5.925e-05 -2.801e-06 -2.217e-04 ...  4.900e+01  1.000e+00  1.000e+00]]
(10000, 15)
[[-1.007e-05  1.371e-06  1.431e-06 ...  4.900e+01  1.000e+00  2.000e+00]
 [-1.395e-05  1.073e-06 -4.411e-06 ...  4.900e+01  1.000e+00  2.000e+00]
 [-1.615e-05 -2.980e-07 -1.347e-05 ...  4.900e+01  1.000e+00  2.000e+00]
 ...
 [-2.730e-05 -8.345e-07  1.729e-06 ...  4.900e+01  1.000e+00  2.000e+00]
 [-3.535e-05 -4.709e-06  1.848e-06 ...  4.900e+01  1.000e+00  2.000e+00]
 [-3.535e-05 -1.007e-05  1.013e-06 ...  4.900e+01  1.000e+00  2.000e+00]]
(10000, 15)
[[ 3.01e-05 -2.38e-07 -1.55e-05 ...  4.90e+01  1.00e+00  3.00e+00]
 [ 1.09e-

(10000, 15)
[[-8.345e-07  3.815e-06  1.192e-06 ...  4.900e+01  3.000e+00  1.000e+00]
 [-5.364e-07  7.033e-06  5.722e-06 ...  4.900e+01  3.000e+00  1.000e+00]
 [-1.788e-07  8.047e-06  1.174e-05 ...  4.900e+01  3.000e+00  1.000e+00]
 ...
 [ 2.950e-05 -6.437e-06  5.484e-06 ...  4.900e+01  3.000e+00  1.000e+00]
 [-9.120e-06  2.265e-06  2.736e-05 ...  4.900e+01  3.000e+00  1.000e+00]
 [-5.931e-05  9.835e-06  6.646e-05 ...  4.900e+01  3.000e+00  1.000e+00]]
(10000, 15)
[[-1.103e-05  6.557e-07 -3.219e-06 ...  4.900e+01  3.000e+00  2.000e+00]
 [-1.609e-05  2.325e-06 -3.040e-06 ...  4.900e+01  3.000e+00  2.000e+00]
 [-1.407e-05  4.172e-06 -1.371e-06 ...  4.900e+01  3.000e+00  2.000e+00]
 ...
 [ 7.272e-06  1.550e-06  2.027e-06 ...  4.900e+01  3.000e+00  2.000e+00]
 [-6.557e-07  1.073e-06  2.503e-06 ...  4.900e+01  3.000e+00  2.000e+00]
 [-4.828e-06  8.941e-07  3.874e-06 ...  4.900e+01  3.000e+00  2.000e+00]]
(10000, 15)
[[-2.259e-05 -2.921e-06  2.682e-06 ...  4.900e+01  3.000e+00  3.000e+00]
 [-

(10000, 15)
[[-1.341e-05 -2.688e-05 -1.385e-04 ...  4.900e+01  4.000e+00  1.000e+00]
 [ 6.467e-05  2.813e-05 -1.415e-04 ...  4.900e+01  4.000e+00  1.000e+00]
 [ 2.060e-04  2.563e-05 -1.051e-04 ...  4.900e+01  4.000e+00  1.000e+00]
 ...
 [-1.907e-06  2.325e-05 -1.192e-06 ...  4.900e+01  4.000e+00  1.000e+00]
 [-2.384e-06  2.712e-05  4.983e-05 ...  4.900e+01  4.000e+00  1.000e+00]
 [-4.232e-06  3.111e-05  1.146e-04 ...  4.900e+01  4.000e+00  1.000e+00]]
(10000, 15)
[[-2.26e-05 -6.56e-07 -4.77e-06 ...  4.90e+01  4.00e+00  2.00e+00]
 [ 1.73e-06 -2.50e-06 -2.09e-06 ...  4.90e+01  4.00e+00  2.00e+00]
 [ 1.77e-05 -4.17e-06  7.75e-07 ...  4.90e+01  4.00e+00  2.00e+00]
 ...
 [-5.44e-05 -5.96e-08 -1.31e-06 ...  4.90e+01  4.00e+00  2.00e+00]
 [-8.01e-05 -1.43e-06 -1.19e-07 ...  4.90e+01  4.00e+00  2.00e+00]
 [-8.51e-05 -7.15e-07  1.01e-06 ...  4.90e+01  4.00e+00  2.00e+00]]
(10000, 15)
[[-1.1015e-04  3.5763e-07 -5.9605e-08 ...  4.9000e+01  4.0000e+00
   3.0000e+00]
 [-1.2708e-04 -3.1590e-06 -1.54

(10000, 15)
[[-2.444e-06 -8.345e-06 -1.562e-05 ...  4.900e+01  6.000e+00  1.000e+00]
 [-1.609e-06 -5.305e-06 -1.025e-05 ...  4.900e+01  6.000e+00  1.000e+00]
 [ 1.073e-06 -3.994e-06 -4.053e-06 ...  4.900e+01  6.000e+00  1.000e+00]
 ...
 [-4.292e-06 -2.444e-06 -2.980e-07 ...  4.900e+01  6.000e+00  1.000e+00]
 [-2.801e-06 -2.742e-06  8.941e-07 ...  4.900e+01  6.000e+00  1.000e+00]
 [-1.788e-06 -4.113e-06  2.205e-06 ...  4.900e+01  6.000e+00  1.000e+00]]
(10000, 15)
[[-7.15e-07  2.68e-06  4.29e-06 ...  4.90e+01  6.00e+00  2.00e+00]
 [-8.34e-07  2.03e-06  5.48e-06 ...  4.90e+01  6.00e+00  2.00e+00]
 [-3.58e-06  2.03e-06  6.32e-06 ...  4.90e+01  6.00e+00  2.00e+00]
 ...
 [-4.29e-06  2.15e-06  1.19e-06 ...  4.90e+01  6.00e+00  2.00e+00]
 [-6.97e-06  2.15e-06  1.79e-07 ...  4.90e+01  6.00e+00  2.00e+00]
 [-5.96e-06  1.13e-06  8.34e-07 ...  4.90e+01  6.00e+00  2.00e+00]]
(10000, 15)
[[ 2.295e-05  2.021e-05  9.358e-06 ...  4.900e+01  6.000e+00  3.000e+00]
 [ 8.166e-06  1.782e-05  9.537e-06 ... 

Train Final Shape: (78400000, 15)


In [20]:
np.unique(train_final[:,-])

(78400000, 15)

In [83]:
test_final_list = []
test_final = np.empty((0,15),dtype=np.float32)
max_samples = 10000

for i in np.unique(test[:, -3]):
    selection = test[test[:, -3] == i]
    for j in [2, 5]:
        new_selection = selection[selection[:, -2] == j]
        for k in np.unique(test[:, -1]):
            k_selection = new_selection[new_selection[:, -1] == k]
            if len(k_selection) >= max_samples:
                data = k_selection[:max_samples]
            else:
                num_samples = len(k_selection)
                padding = np.zeros((max_samples - num_samples, 15), dtype=np.float32)
                data = np.vstack((k_selection, padding))
            test_final_list.append(data)

test_final = np.vstack(test_final_list)

print("Test Final Shape:", test_final.shape)

Test Final Shape: (39200000, 15)


In [131]:
train_final = train_final[:,:-1]
test_final = test_final[:,:-1]

In [132]:
train_final[:,-2], test_final[:,-2]

(array([ 1.,  1.,  1., ..., 49., 49., 49.], dtype=float32),
 array([ 1.,  1.,  1., ..., 49., 49., 49.], dtype=float32))

In [133]:
train_final.shape, test_final.shape

((78400000, 14), (39200000, 14))

In [134]:
np.unique(train_final[:,-1]), np.unique(test_final[:,-1])

(array([0., 1., 3., 4., 6.], dtype=float32),
 array([0., 2., 5.], dtype=float32))

In [146]:
Train_Class_Label = train_final[:,12]
Test_Class_Label = test_final[:,12]

In [147]:
train_final = train_final[:,:-1]
test_final = test_final[:,:-1]

In [150]:
train_final = train_final[:,:-1]
test_final = test_final[:,:-1]

In [151]:
train_final.shape , test_final.shape

((78400000, 12), (39200000, 12))

In [152]:
np.savetxt("Train_Class_Label.csv", Train_Class_Label , delimiter = ',', fmt='%f')
np.savetxt("Test_Class_Label.csv", Test_Class_Label, delimiter = ',', fmt='%f')
np.savetxt("Train.csv", train_final, delimiter = ',', fmt='%f')
np.savetxt("Test.csv", test_final, delimiter = ',', fmt='%f')